In [3]:
#Dependencies 
import pandas as pd 
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
import requests
import json
import time
import csv

In [4]:
# File to Load
import csv
load_file = "nba_ticket_prices.csv"
nba_ticket_prices = pd.read_csv(load_file, encoding= 'unicode_escape')
nba_ticket_prices.head()

,team,Average Ticket Price,Arena,lat,long
0,76ers,$81.00,Wells Fargo Center,39.9012,-75.1720
1,Blazers,$92.00,Moda Center,45.5316,-122.6670
2,Bucks,$60.00,Fiserv Forum,43.0451,-87.9174
3,Bulls,$165.00,United Center,41.8807,-87.6742
4,Cavaliers,$42.00,Rocket Mortgage FieldHouse,41.4966,-81.6881


In [5]:
## create engine
engine = create_engine('sqlite:///sports.db', echo = True)
sqlite_connection = engine.connect()

2020-10-17 16:23:12,858 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-17 16:23:12,862 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:23:12,863 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-17 16:23:12,864 INFO sqlalchemy.engine.base.Engine ()


In [6]:
Base=automap_base()
Base.prepare(sqlite_connection, reflect=True)
Base.classes.keys()

2020-10-17 16:23:17,841 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-10-17 16:23:17,842 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:23:17,845 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("mlb_ticket_price")
2020-10-17 16:23:17,846 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:23:17,849 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'mlb_ticket_price' AND type = 'table'
2020-10-17 16:23:17,851 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:23:17,853 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("mlb_ticket_price")
2020-10-17 16:23:17,855 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:23:17,856 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("mlb_ticket_price")
2020-10-17 16:23:17,858 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:23:17,859 INFO sqlalchemy.engine

['mlb_ticket_price', 'nba_ticket_price', 'nfl_ticket_price']

In [7]:
#Set name of table to create
nba_tickets = "nba_ticket_prices"
nba_ticket_prices.to_sql(nba_tickets, sqlite_connection, if_exists = 'fail')

2020-10-17 16:24:03,827 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("nba_ticket_prices")
2020-10-17 16:24:03,829 INFO sqlalchemy.engine.base.Engine ()


ValueError: Table 'nba_ticket_prices' already exists.

In [44]:
#Putting primary key into ticket price table
create_query_sql='create table nba_ticket_price (\
id_num bigint primary key,\
team text,\
Average Ticket Price bigint,\
Arena text,\
lat double,\
long double\
);'
sqlite_connection.execute(create_query_sql)

2020-10-17 16:12:06,925 INFO sqlalchemy.engine.base.Engine create table nba_ticket_price (id_num bigint primary key,team text,Average Ticket Price bigint,Arena text,lat double,long double);
2020-10-17 16:12:06,927 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:12:06,933 INFO sqlalchemy.engine.base.Engine COMMIT


In [45]:
sqlite_connection.execute('insert into nba_ticket_price select * from nba_ticket_prices')

2020-10-17 16:12:28,788 INFO sqlalchemy.engine.base.Engine insert into nba_ticket_price select * from nba_ticket_prices
2020-10-17 16:12:28,789 INFO sqlalchemy.engine.base.Engine ()
2020-10-17 16:12:28,792 INFO sqlalchemy.engine.base.Engine COMMIT


In [8]:
#Make fetch happen, check work 
sqlite_connection.execute('select * from nba_ticket_price').fetchall()

2020-10-17 16:24:13,026 INFO sqlalchemy.engine.base.Engine select * from nba_ticket_price
2020-10-17 16:24:13,028 INFO sqlalchemy.engine.base.Engine ()


[(0, '76ers', '$81.00 ', 'Wells Fargo Center', 39.9012, -75.172),
 (1, 'Blazers', '$92.00 ', 'Moda Center', 45.5316, -122.667),
 (2, 'Bucks', '$60.00 ', 'Fiserv Forum', 43.0451, -87.9174),
 (3, 'Bulls', '$165.00 ', 'United Center', 41.8807, -87.6742),
 (4, 'Cavaliers', '$42.00 ', 'Rocket Mortgage FieldHouse', 41.4966, -81.6881),
 (5, 'Celtics', '$85.00 ', 'TD Garden', 42.3662, -71.0621),
 (6, 'Clippers', '$130.00 ', 'Staples Center', 34.043, -118.26700000000001),
 (7, 'Grizzlies', '$55.00 ', 'FedExForum', 35.1381, -90.0506),
 (8, 'Hawks', '$111.00 ', 'State Farm Arena', 33.7573, -84.3963),
 (9, 'Heat', '$155.00 ', 'American Airlines Arena', 25.7814, -80.187),
 (10, 'Hornets', '$89.00 ', 'Spectrum Center', 35.2252, -80.8394),
 (11, 'Jazz', '$76.00 ', 'Vivint Smart Home Arena', 40.7683, -111.90100000000001),
 (12, 'Kings', '$75.00 ', 'Golden 1 Center', 38.5802, -121.5),
 (13, 'Knicks', '$200.00 ', 'Madison Square Garden', 40.7505, -73.9934),
 (14, 'Lakers', '$180.00 ', 'Staples Center', 